<h1> Model Iteration 1 for Movie Sentiment Analysis </h1>
<h2> Yuzhong Huang & Wilson Tang </h2>
<h3> Data Description
</h3>
<p>
"There's a thin line between likably old-fashioned and fuddy-duddy, and The Count of Monte Cristo ... never quite settles on either side."
The Rotten Tomatoes movie review dataset is a corpus of movie reviews used for sentiment analysis, originally collected by Pang and Lee. In their work on sentiment treebanks, Socher et al. used Amazon's Mechanical Turk to create fine-grained labels for all parsed phrases in the corpus. This competition presents a chance to benchmark your sentiment-analysis ideas on the Rotten Tomatoes dataset. You are asked to label phrases on a scale of five values: negative, somewhat negative, neutral, somewhat positive, positive. Obstacles like sentence negation, sarcasm, terseness, language ambiguity, and many others make this task very challenging.
</p>

<h1>
Exploration Journal Debrief
</h1>
<p>
We first followed the bag of words kaggle tutorial(https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-1-for-beginners-bag-of-words) to create a (256060, 1000) bag of words as a feature map, and applied random forest classification to fit our trainning data. Due to limitedness of memory, we were only able to perform random forest classification at tree level of 10. And we achieved score of 0.56750 in Kaggle.
We then used scikit's naive bayesian model and explored using 
    - tfidf 
    - simple frequencies 
    - our own model(started)
Naive Bayesian appeared to work better and we will be further working on improving it later before moving onto more techniques
</p>

In [1]:
import random

import pandas as pd
from pandas import Series,DataFrame
import thinkstats2
import thinkplot
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from textblob import TextBlob
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

%matplotlib inline
nltk.download('stopwords')

/home/yuzhong/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/home/yuzhong/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/yuzhong/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# get train & test csv files as a DataFrame
train = pd.read_csv("train.tsv", sep='\t')
test = pd.read_csv("test.tsv", sep='\t')

# drop nan data in train
train = train.dropna()
test = test.dropna()

In [3]:
def review_to_words(review):
    """ This function will take in a loaded review as well as a list of stop words and process it by
    removing non letters, converting to lowercase and splitting, removing stopwords, and 
    returning a reconstructed review
    """
    letters_only = re.sub("[^a-zA-Z]", " ", review)
    words = letters_only.lower().split()
    stops = set(stopwords.words("english"))
    meaningful_words = [w for w in words if not w in stops]
    return ( " ".join(meaningful_words ))

def add_processed_review(df):
    """ add_processed_review takes a dataframe and process the phrases within the dataframe and 
    add a new column "ProcessedReview".
    """
    df['ProcessedReview'] = df['Phrase'].apply(review_to_words)
    return df

###################################################################################################

def get_word_bag(vect, data):
    """
    Takes a list of text data and a vectorizer to transform the data to a bag of words in list
    """
    bag = vect.fit_transform(data)
    return bag.toarray()

def nb_model(train_features, test_features, target):
    """
    Return the predictions from navie baysian trained from given features and target
    """
    nb = MultinomialNB().fit(train_features, target)
    return nb.predict(test_features)

def submission(predictions):
    """
    Use pandas to write the comma-separated output 
    file based on the predictions
    """
    
    submission = pd.DataFrame({
            "PhraseId": test["PhraseId"],
            "Sentiment": predictions
        })

    submission.to_csv("Movie_Sentiment_Analysis_Submission.csv", index=False, quoting=3)

###################################################################################################
    
def add_whole_sentence(df):
    """
    Generates a boolean 'WholeSentence' column for given dataset
    to indicate whether the phrase is a whole sentence or not
    """
    df['WholeSentence'] = False
    # initialize a last_id for comparison
    last_id = 0
    # loop throught the dataframe to set the first sentence for each sentence group's 'WholeSentence' to be True
    for index, row in df.iterrows():
        # changes 'WholeSentence' to be true for the first sentence for each sentence group
        if last_id != row['SentenceId']:
            last_id = row['SentenceId']
            df.set_value(index, 'WholeSentence', True)
    return df 

In [4]:
train = add_processed_review(train)
test = add_processed_review(test)

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 1000) 

train_feature = get_word_bag(vectorizer, train['ProcessedReview'].tolist())
test_feature = get_word_bag(vectorizer, test['ProcessedReview'].tolist())
submission(nb_model(train_feature, test_feature, train['Sentiment']))

<h1>
    Naive Bayesian By Hand
</h1>
<p> 
    Now we will try to implement our naive bayesian then compare it to scikit learns algorithm.
</p>

<p>
    We have 5 different potential reviews: 0,1,2,3,4 and will need 5 class probabilities to represent each potential outcome. We will get the frequency per word and then calculate the probability for each of the class probabilites by dividing the frequency of word with the total number of occurences that word occurs in that review. To get the probability of a phrase or sentence we multiply the probability for each word in that phrase against he probability of any document that expresses that sentiment.
</p>

In [5]:
# we will start by creating 5 probabilities for any given document that expresses that sentiment
doc_prob_0 = float(len(train[train.Sentiment == 0]))/len(train)
doc_prob_1 = float(len(train[train.Sentiment == 1]))/len(train)
doc_prob_2 = float(len(train[train.Sentiment == 2]))/len(train)
doc_prob_3 = float(len(train[train.Sentiment == 3]))/len(train)
doc_prob_4 = float(len(train[train.Sentiment == 4]))/len(train)

doc_probs = [doc_prob_0, doc_prob_1, doc_prob_2, doc_prob_3, doc_prob_4]

In [6]:
def sentiment_word(df):
    """ Takes: Test data in the form of dataframe
        Returns: A list value of words number in different sentiment
    """
    sentiment_word = np.array([0, 0, 0, 0, 0])
    for index, row in df.iterrows():
        sentiment = row["Sentiment"]
        sentiment_word[sentiment] += (row["ProcessedReview"].count(" ") + 1)
    return sentiment_word

def create_reference_histogram(df):
    """Takes: A dataset of strings
       Returns: Dictionary w/ keys-words, value- list of values[sent=0,sent=1,sent=2,sent=3,sent=4,total] 
    """
    histogram = {}
    #we will iterate through each sentence and word and increment the index of the list of values
    for index, row in df.iterrows():
        sentiment_value = df.Sentiment[index]
        words = df["ProcessedReview"][index].split()
        for word in words:
            if word in histogram:
                histogram[word][5] +=1
                histogram[word][sentiment_value] += 1
            else:
                histogram[word]= np.array([0,0,0,0,0,1])
                histogram[word][sentiment_value] += 1
    return histogram  

def calc_sentiment_probabilities(ref_dict,sent_freq):
    """ Takes: A sentiment value you are trying to test for
        Returns: Dictionary w/ keys-word value-probability of that word matching the value
        frequency = matching_value_freq/total_freq
    """   
    # make a copy of ref_dict
    final_dict = {}
    # we will iterate through every word in the dictionary and change the values from a list to the probability we want 
    # if the value is 0 we will make probability 1 as part of smooth
    for word in ref_dict:
        final_dict[word] = ref_dict[word][:-1]/(sent_freq + float(ref_dict[word][5]))
    return final_dict

def smooth(lst):
    for ele in lst:
        if ele == 0: ele = 1
    return lst

def phrase_sentiment_prob(text, refer):
    """ Takes: A phrase in the form of a string
        Returns: A list of probabilities for different sentiments
    """
    probs = np.asarray(doc_probs)
    for word in text.split():
        if word in refer:
            probs *= smooth(refer[word])
    return probs    

def make_predictions(data, refer):
    """ Takes: Test data in the form of dataframe
        Returns: Submission File w. Sentiment value calculated
    """
    result = []
    lst = []
    for index, row in data.iterrows():
        lst = phrase_sentiment_prob(data['ProcessedReview'][index], refer).tolist()
        result.append(lst.index(max(lst)))
    return result

In [7]:
overall_sent = sentiment_word(train)
hist = create_reference_histogram(train)
reference_hist = calc_sentiment_probabilities(hist,overall_sent)

In [8]:
sentiment_word(train)

array([ 45485, 136635, 239989, 156156,  54868])

In [9]:
result = make_predictions(test, reference_hist)

In [10]:
submission(result)

## A Single Layer Neural Network

In [33]:
class Network(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in xrange(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in xrange(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print "Epoch {0}: {1} / {2}".format(
                    j, self.evaluate(test_data), n_test)
            else:
                print "Epoch {0} complete".format(j)

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in xrange(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

#### Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

def reshape_list(lst):
    """Reshape dataset into dimension 1 arrays"""
    for i in range(len(lst)):
        tup_l = list(lst[i])
        tup_l[0] = tup_l[0].reshape((1000, 1))
        tup_l[1] = tup_l[1].reshape((1, 1))
        lst[i] = tuple(tup_l)    
    return lst

In [57]:
net = Network([1000, 200, 100, 80, 60, 50, 20, 5])
val = train['Sentiment'].reshape((156060, 1))
train_set = reshape_list(zip(train_feature, val))
net.SGD(train_set[:1000], 30, 10, .1, test_data=train_set[1000:1200])

Epoch 0: 52 / 200
Epoch 1: 88 / 200
Epoch 2: 89 / 200
Epoch 3: 91 / 200
Epoch 4: 91 / 200
Epoch 5: 91 / 200
Epoch 6: 93 / 200
Epoch 7: 93 / 200
Epoch 8: 93 / 200
Epoch 9: 93 / 200
Epoch 10: 93 / 200
Epoch 11: 91 / 200
Epoch 12: 92 / 200
Epoch 13: 92 / 200
Epoch 14: 92 / 200
Epoch 15: 92 / 200
Epoch 16: 92 / 200
Epoch 17: 90 / 200
Epoch 18: 90 / 200
Epoch 19: 90 / 200
Epoch 20: 89 / 200
Epoch 21: 89 / 200
Epoch 22: 89 / 200
Epoch 23: 89 / 200
Epoch 24: 89 / 200
Epoch 25: 89 / 200
Epoch 26: 89 / 200
Epoch 27: 88 / 200
Epoch 28: 88 / 200
Epoch 29: 88 / 200
